In [ ]:
import json
from collections import defaultdict
from datetime import datetime

# Ruta al archivo de log (ajústala según sea necesario)
log_file_path = "datos.json"

# Diccionario para almacenar información por sesión
sessions = defaultdict(lambda: {
    "StartTracking": None,
    "SessionStart": None,
    "LeftEqEvent": [],
    "RightEqEvent" : [],
    "StartGame": [],
    "EnemyAppeared": [],
    "EnemyShot": [],
    "BulletShot": [],
    "ButtonAppearedEvent": [],
    "ButtonPressedEvent": [],
    "MaximumSequenceEvent" : [],
    "EndGame": [],
    "SessionEnd": None,
    "StopTracking": None
})

def convertir_timestamp(timestamp):
    return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

with open(log_file_path, "r") as file:
    for line in file:
        entry = json.loads(line.strip())
        session_id = entry["mSessionID_"]
        event = entry["_data"]["Event"]
        timestamp = entry["_timeStamp"]

        # Clasificamos los eventos por tipo
        if event == "StartTracking":
            sessions[session_id]["StartTracking"] = convertir_timestamp(timestamp/1000)
        elif event == "SessionStart":
            sessions[session_id]["SessionStart"] = convertir_timestamp(timestamp/1000)
        elif event == "LeftEq":
            sessions[session_id]["LeftEqEvent"].append(entry["_data"])  
        elif event == "RightEq":
            sessions[session_id]["RightEqEvent"].append(entry["_data"])
        elif event.startswith("StartGame"):
            sessions[session_id]["StartGame"].append({"type": event, "time": convertir_timestamp(timestamp/1000)})
        elif event == "EnemyAppeared":
            sessions[session_id]["EnemyAppeared"].append(entry["_data"])
        elif event == "EnemyShot":
            sessions[session_id]["EnemyShot"].append(entry["_data"])
        elif event == "ShootBullet":
            sessions[session_id]["BulletShot"].append(entry["_data"])
        elif event.startswith("EndGame"):
            sessions[session_id]["EndGame"].append({"type": event, "time": convertir_timestamp(timestamp/1000)})
        elif event == "SessionEnd":
            sessions[session_id]["SessionEnd"] = convertir_timestamp(timestamp/1000)
        elif event == "StopTracking":
            sessions[session_id]["StopTracking"] = convertir_timestamp(timestamp/1000)
        elif event == "ButtonAppearedEvent":
            sessions[session_id]["ButtonAppearedEvent"].append(entry["_data"])
        elif event == "ButtonPressedEvent":
            sessions[session_id]["ButtonPressedEvent"].append(entry["_data"])
        elif event == "MaximumSequenceEvent":
            sessions[session_id]["MaximumSequenceEvent"].append(entry["_data"])

# Mostrar resumen por sesión
d = 0
for session_id, data in sessions.items():
    print(f"\n---------------- Trace NUMBER: {d} ----------------")
    print(f"Session ID: {session_id}")
    print(f"StartTracking: {data['StartTracking']}")
    print(f"SessionStart: {data['SessionStart']}")
    for game in data["StartGame"]:
        print(f"Game Started: {game['type']} at {game['time']}")
    print(f"Enemies Appeared: {len(data['EnemyAppeared'])}")
    print(f"Enemies Shot: {len(data['EnemyShot'])}")
    print(f"Bullets shot: {len(data['BulletShot'])} ")
    #for i, left_eq in enumerate(data["LeftEqEvent"]):
    for left_eq_event in enumerate(data["LeftEqEvent"]):
        print("Contenido LeftEqEvent:", left_eq_event)

    for right_eq_event in enumerate(data["RightEqEvent"]):
        print("Contenido RightEqEvent:", right_eq_event)

    print(f"SessionEnd: {data['SessionEnd']}")
    print(f"StopTracking: {data['StopTracking']}")
    d += 1

deltas_por_sesion = defaultdict(list)
for session_id, data in sessions.items():
    apariciones = data["EnemyAppeared"]
    muertes = data["EnemyShot"]

    # Creamos diccionarios por ID
    aparicion_por_id = {}
    for evento in apariciones:
        id_enemigo = evento.get("Enemy id: ")
        tiempo = evento.get("Enemy appeared at time: ") 
        if id_enemigo is not None and tiempo != 0:
            aparicion_por_id[id_enemigo] = float(tiempo.replace(",", "."))

    for evento in muertes:
        id_enemigo = evento.get("Enemy id: ")
        tiempo_str = evento.get("Enemy killed at time: ")
        tiempo_aparicion_str = aparicion_por_id.get(id_enemigo)

        if id_enemigo is not None and tiempo_str and tiempo_aparicion_str:
            try:
                tiempo_muerte = float(tiempo_str.replace(",", "."))
                tiempo_aparicion = tiempo_aparicion_str
                delta = tiempo_muerte - tiempo_aparicion
                if delta >= 0:
                    deltas_por_sesion[session_id].append(delta)
            except ValueError:
                continue  # Por si acaso alguna conversión falla
 
output_md_path = "resumen_sesiones.md"

with open(output_md_path, "w", encoding="utf-8") as md_file:
    d = 0
    for session_id, data in sessions.items():
        md_file.write(f"# 🔹 Traza {d}. Sesión `{session_id}`\n\n")
        md_file.write(f"- **Comienzo de trackeo:** {data['StartTracking']}\n")
        # md_file.write(f"- **SessionStart:** {data['SessionStart']}\n\n")

        md_file.write(f"- **Datos audiometría:**\n\n")
        for i, left_eq_event in enumerate(data["LeftEqEvent"]):
            md_file.write(f"  -  **Ecualización IZQUIERDA:** {left_eq_event.get('lefteq', 'Clave no encontrada')}\n")

        for j, right_eq_event in enumerate(data["RightEqEvent"]):
            md_file.write(f"  -  **Ecualización DERECHA:** {right_eq_event.get('righteq', 'Clave no encontrada')}\n")

        for start_game, end_game in zip(data["StartGame"], data["EndGame"]):
            if start_game['type'] == 'StartGameFPS':
                md_file.write(f"- Se ha **empezado** a jugar `{start_game['type']}` en el tiempo `{start_game['time']}`\n")
                md_file.write(f"    -   **Enemigos spawneados:** {len(data['EnemyAppeared'])}\n")
                md_file.write(f"    - **Enemigos abatidos:** {len(data['EnemyShot'])}\n")
                if len(data['BulletShot']) > 0:
                    md_file.write(f"    - **Disparos totales:** {len(data['BulletShot'])}\n")
                    md_file.write(f"    - **Precisión** : {((len(data['EnemyShot']) / len(data['BulletShot'])) * 100):.2f} %\n")
                deltas = deltas_por_sesion.get(session_id, [])
                if deltas:
                    media = sum(deltas) / len(deltas)
                    md_file.write(f"    -  **Tiempo medio en matar enemigos**: {media:.2f} segundos\n")
                md_file.write(f"- Se ha **terminado** el juego de `{end_game['type']}` en el tiempo `{end_game['time']}`\n")
            elif start_game['type'] == 'MEMORY':
                md_file.write(f"  - Se ha **empezado** a jugar `{game['type']}` en el tiempo `{game['time']}`\n")
                md_file.write(f"    -   **Botones spawneados:** {len(data['ButtonAppearedEvent'])}\n")
                md_file.write(f"    -   **Botones presionados:** {len(data['ButtonPressedEvent'])}\n")
                deltas = deltas_por_sesion.get(session_id, [])
                if deltas: 
                    media = sum(deltas) / len(deltas)
                    md_file.write(f"   -  Tiempo medio en presionar botones: {media:.2f} segundos\n")
                md_file.write(f"- Se ha **terminado** el juego de `{end_game['type']}` en el tiempo `{end_game['time']}`\n ")
             
        # md_file.write(f"\n- **SessionEnd:** {data['SessionEnd']}\n")
        md_file.write(f"- **Cese de trackeo:** {data['StopTracking']}\n")
        md_file.write(f"\n---\n\n")
        d += 1

print(f"\n Resumen guardado en formato Markdown en '{output_md_path}'")


---------------- Trace NUMBER: 0 ----------------
Session ID: 758d3227-3368-43be-be74-34f848f914e2
StartTracking: 2025-05-12 19:02:59
SessionStart: 2025-05-12 19:02:59
Game Started: StartGameFPS at 2025-05-12 19:03:04
Game Started: StartGameFPS at 2025-05-12 19:04:50
Enemies Appeared: 27
Enemies Shot: 16
Bullets shot: 0 
SessionEnd: 2025-05-12 19:07:04
StopTracking: 2025-05-12 19:07:04

---------------- Trace NUMBER: 1 ----------------
Session ID: 0bdcec73-94a1-4a6c-abae-ceb43c55b161
StartTracking: 2025-05-12 19:07:52
SessionStart: 2025-05-12 19:07:52
Game Started: StartGameFPS at 2025-05-12 19:08:00
Enemies Appeared: 6
Enemies Shot: 0
Bullets shot: 0 
SessionEnd: 2025-05-12 19:09:39
StopTracking: 2025-05-12 19:09:39

---------------- Trace NUMBER: 2 ----------------
Session ID: 7a5f75ff-116d-48a2-a107-3dcf3504aeb4
StartTracking: 2025-05-12 19:09:57
SessionStart: 2025-05-12 19:09:57
Enemies Appeared: 0
Enemies Shot: 0
Bullets shot: 0 
SessionEnd: 2025-05-12 19:10:12
StopTracking: 2025